In [2]:
import pandas as pd
import pyodbc
import pandas_profiling

# setting up database connection strings

In [1]:
def sqlserver_sql(q, server = 'SQL2017', db = 'IPL_IRASV6_STAGE'):
    driver = '{SQL Server Native Client 11.0}'
    conn = pyodbc.connect("Driver="+driver+";Server="+server+";Database="+db+";Trusted_Connection=yes;")

    # performing query to database
    df = pd.read_sql_query(q,conn)

    conn.close()
    return df

# the following Pandas groupby will:
    # 1. Group the dataframe based on BeginStationSeriesId and EndStationSeriesId
    # 2. Apply a calculation to subtract the current record EndStationNum from the next BeginStationNum
    # Improvement might be to convert this to chainage, group by lineloopID, and repeat this with chaingages instead
    # Another improvement might be to also gather MLVCorrection records, or StationSeries records, and also check that the observed data covers entire pipeline

In [5]:
q0 = """select ld.code [type], ca.* from classArea ca
        inner join listdomain ld on ca.ClassAreaRatingDomainId = ld.id"""

df1 = sqlserver_sql(q0)
df1.head()

df1 = df1.sort_values(by=['BeginStationSeriesId','BeginStationNum']).reset_index(drop=True)
df1

ss_df = df1[['BeginStationSeriesId','BeginStationNum','EndStationSeriesId','EndStationNum']]
ss_df.head()

# ss_df.groupby(['BeginStationSeriesId','EndStationSeriesId']).apply(lambda x: x['BeginStationNum'].shift(-1)-x['EndStationNum']).fillna(0.00)
# df1.groupby(['BeginStationSeriesId','EndStationSeriesId']).apply(lambda x: x['BeginStationNum'].shift(-1)-x['EndStationNum']).fillna(0.00)

ss_df_overlaps = ss_df.groupby(['BeginStationSeriesId','EndStationSeriesId']).apply(lambda x: x['BeginStationNum'].shift(-1)-x['EndStationNum']).reset_index(name='record_diff').fillna(0.00)
# ss_df_overlaps.query('record_diff!=0.')
df1.join(df1.groupby(['BeginStationSeriesId','EndStationSeriesId']).apply(lambda x:  x['BeginStationNum'].shift(-1)-x['EndStationNum']).reset_index(name='record_diff').fillna(0.00).loc[:,'record_diff']).loc[[691,692],:]#.query("record_diff!=0.")

# ss_df[ss_df.index.isin(ss_df_overlaps.query('record_diff!=0.').index)]

,type,Id,EffectiveStartDate,EffectiveEndDate,DwellingUnitCnt,ClassAreaRatingDomainId,BeginStationSeriesId,BeginStationNum,EndStationSeriesId,EndStationNum,CFRSpecialCaseFlag,RCScenarioID,LastModByUserId,LastModDateTime,SourceId,CreatedDate,CreatedUser,record_diff
691,1 with no dwellings,671.0,None,None,None,205.0,117.0,10569.27234,117.0,16862.86954,None,None,DYNAMIC_RISK\DataExchangeUser,2019-10-10 19:01:45.543,671,2019-07-30 08:27:11.330,DYNAMIC_RISK\DataExchangeUser,0.045
692,1 with no dwellings,660.0,None,None,None,205.0,117.0,16862.91454,117.0,17322.50955,None,None,DYNAMIC_RISK\DataExchangeUser,2019-10-10 19:01:45.543,660,2019-07-30 08:27:11.330,DYNAMIC_RISK\DataExchangeUser,0.000


# Replicating same exercise above, but this time doing it with MLVCorrection, and converting the linear table's StationNums into chainages

In [6]:
def check_table_overlap(table, s="SQL2017", d="IPL_IRASV6_STAGE"):
    # query for table to check for overlaps
    q1 = f"""set nocount on;
            select e.Id [RecordId],
            (mlv1.MultiplierNum*e.BeginStationNum+mlv1.FactorNum) [eBeginChainage],
            (mlv2.MultiplierNum*e.EndStationNum+mlv2.FactorNum) [eEndChainage], e.* from {table} e
            join MlvCorrection mlv1 on mlv1.StationSeriesId = e.BeginstationSeriesId
            join MlVcorrection mlv2 on mlv2.StationseriesId = e.EndStationSeriesId"""

    df2 = sqlserver_sql(q1, server=s, db=d)

    # sorting in increasing stationseriesid and chainage
    df2 = df2.sort_values(by=['BeginStationSeriesId','eBeginChainage']).reset_index(drop=True)

    # creating subset containing only the chainages
#     ss_df2 = df2[['BeginStationSeriesId','eBeginChainage','EndStationSeriesId','eEndChainage']]

    # following pandas statement displays the subtraction from one record's endchainage to the next records beginchainage
#     ss_df2.groupby(['BeginStationSeriesId','EndStationSeriesId']).apply(lambda x: x['eBeginChainage'].shift(-1)-x['eEndChainage']).fillna(0.00)

    # following statements filter out anything that doesn't have an overlap
#     ss_df2_overlaps = ss_df2.groupby(['BeginStationSeriesId','EndStationSeriesId']).apply(lambda x: x['eBeginChainage'].shift(-1)-x['eEndChainage']).reset_index(name='record_diff').fillna(0.00)
    
    df2 = df2.join(df2.groupby(['BeginStationSeriesId','EndStationSeriesId']).apply(lambda x:  x['eBeginChainage'].shift(-1)-x['eEndChainage']).reset_index(name='record_diff').fillna(0.00).record_diff)
    return df2[df2.columns.values[[0,1,2,-1]+[x for x in range(3,len(df2.columns)-1)]]]#.query("record_diff != 0.")

# Following section checks whether a particular table contains records beyond  the bounds of the StationSeries 

In [7]:
def check_table_span(table, s="SQL2017", d="IPL_IRASV6_STAGE"):
    # query for the table to check if records fall within bounds of stationseries
    q2 = f"""set nocount on;
            select ss.lineloopid [LineLoopId],
            (mlv1.MultiplierNum*e.BeginStationNum+mlv1.FactorNum) [eBeginChainage],
            (mlv2.MultiplierNum*e.EndStationNum+mlv2.FactorNum) [eEndChainage], e.* from {table} e
            join MlvCorrection mlv1 on mlv1.StationSeriesId = e.BeginstationSeriesId
            join MlVcorrection mlv2 on mlv2.StationseriesId = e.EndStationSeriesId
            join stationseries ss on e.Beginstationseriesid = ss.id
            """

    q3 = f"""set nocount on;
            select 
            (mlv.MultiplierNum*ss.BeginStationNum+mlv.FactorNum) [sBeginChainage],
            (mlv.MultiplierNum*ss.EndStationNum+mlv.FactorNum) [sEndChainage], ss.* from stationseries ss
            join MlvCorrection mlv on mlv.StationSeriesId = ss.id
            """

    df3 = sqlserver_sql(q2, server=s, db=d)
    df4 = sqlserver_sql(q3, server=s, db=d)

    # this next statement just aggregates the event table data and shows the span of the data. DOESN'T PICK OUT GAPS.
    # df3.groupby("LineLoopId").agg({'eBeginChainage':'min','eEndChainage':'max'})


    return df4.groupby("LineLoopId").agg({'sBeginChainage':'min','sEndChainage':'max'}).join(df3.groupby("LineLoopId").agg({'eBeginChainage':'min','eEndChainage':'max'})).assign(delta_end = lambda x: x.sEndChainage-x.eEndChainage,
                                                                                                                                                                                    delta_begin = lambda x: x.sBeginChainage-x.eBeginChainage).query("(delta_end!=0.) | (delta_begin!=0.)")

In [8]:
IRAS_table = "maop"

In [10]:
temp1_stg = check_table_overlap(IRAS_table)
temp1_stg = temp1_stg.loc[temp1_stg.query("record_diff != 0.").index.append(temp1_stg.query("record_diff != 0.").index+1).drop_duplicates(),:].sort_index()
display(temp1_stg.iloc[:,[0,1,2,3]].shape)

temp1_tu = check_table_overlap(IRAS_table,)
temp1_tu = temp1_tu.loc[temp1_tu.query("record_diff != 0.").index.append(temp1_tu.query("record_diff != 0.").index+1).drop_duplicates(),:].sort_index()
display(temp1_tu.iloc[:,[0,1,2,3]].shape)

temp2_stg = check_table_span(IRAS_table)
display(temp2_stg.query("~delta_end.isnull() | ~delta_begin.isnull()").shape)

temp2_tu = check_table_span(IRAS_table)
display(temp2_tu.query("~delta_end.isnull() | ~delta_begin.isnull()").shape)

(9, 4)

(9, 4)

(23, 6)

(23, 6)

In [12]:
print('STAGE')
print(f'\n{temp1_stg.iloc[:,[0,1,2,3]].shape[0]} findings in event table containing gaps, overlaps, or duplicates')
print(f'\n{temp2_stg.query("~delta_end.isnull() | ~delta_begin.isnull()").shape[0]} LineLoop records with data, and problems with event data gaps, or out of bounds')
print(f'\n{temp2_stg.query("eBeginChainage.isnull()").shape[0]} LineLoop records with no data in the event table')

display(temp1_stg.iloc[:,[0,1,2,3]])
display(temp2_stg.query("~delta_end.isnull() | ~delta_begin.isnull()"))
display(temp2_stg.query("eBeginChainage.isnull()").index.to_series())

STAGE

9 findings in event table containing gaps, overlaps, or duplicates

23 LineLoop records with data, and problems with event data gaps, or out of bounds

0 LineLoop records with no data in the event table


,RecordId,eBeginChainage,eEndChainage,record_diff
234,796.0,0.00000,15959.23051,0.045
235,803.0,15959.27551,16418.87052,0.000
728,572.0,0.00000,2266.22307,0.011
729,573.0,2266.23407,4401.44014,0.000
732,230.0,0.00000,985.20803,-0.011
733,231.0,985.19703,7728.28424,-0.177
734,768.0,7728.10724,14434.31146,0.000
749,790.0,0.00000,2676.83808,0.002
750,791.0,2676.84008,2790.13809,0.000


,sBeginChainage,sEndChainage,eBeginChainage,eEndChainage,delta_end,delta_begin
LineLoopId,,,,,,
59.0,0.0,33463.59107,0.00000,33463.58807,0.00300,0.00000
67.0,0.0,25160.36080,0.00000,25160.35780,0.00300,0.00000
68.0,0.0,53365.33171,0.00000,53365.32771,0.00400,0.00000
76.0,0.0,20402.06265,0.00000,20402.06165,0.00100,0.00000
118.0,0.0,47775.39253,0.00000,47775.38853,0.00400,0.00000
120.0,0.0,7866.49025,0.00000,7866.48625,0.00400,0.00000
122.0,0.0,82625.03264,12383.61639,58833.70388,23791.32876,-12383.61639
124.0,0.0,58834.24188,0.00000,58834.23988,0.00200,0.00000
139.0,0.0,10355.69833,0.00000,10355.69633,0.00200,0.00000


Series([], Name: LineLoopId, dtype: float64)

In [427]:
%pwd

'N:\\Python\\EEC_Findings'